In [1]:
!pip install langchain google-generativeai python-dotenv langchain_community langchain-google-genai faiss-cpu -q

In [29]:
import os #trabajar arhivoc de texto
from dotenv import load_dotenv
from langchain.document_loaders import TextLoader # Cargar archivo de texto
from langchain.text_splitter import CharacterTextSplitter # Dividir textos en textos mas chicos
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Trnasformar en vectores los textos 
from langchain.vectorstores import FAISS #Buscar en forma de vectores

from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain.chains import RetrievalQA

In [30]:
#Cargar y procesar el documento

loader = TextLoader('gastos.txt', encoding='utf-8')
documents = loader.load()

In [31]:
#Dividir el texto en chunks

text_splitter= CharacterTextSplitter(chunk_size=500, chunk_overlap=50) # Splitter recursivo
texts = text_splitter.split_documents(documents)

In [32]:
#Crear embeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.getenv("GENAI_API_KEY")
)

In [33]:
#Crear base de datos vectorial

db = FAISS.from_documents(texts, embeddings)

In [44]:
#Realizar una busuqeda

query = "Hola chat como estas? te pregunto ¿en que gasto mas mi dinero?."

# Inicializar el modelo de chat
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    google_api_key=os.getenv("GENAI_API_KEY"),
    temperature=0.7 #Creatividad del modelo
)

# Crear la cadena de recuperación y respuesta
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

# Ejecutar la consulta y obtener la respuesta
response = qa_chain.run(query)


# Reemplazar los asteriscos por negrita en Markdown
response = response.replace("* **", "**").replace("**:", "**").replace("*", "")

# Imprimir la respuesta
print(response)

Hola! Según la información proporcionada, gastas más dinero en "Arriendo" con un monto de 1200000.
